# Summary of Special Class Members

The built-in python stuff like str() and len() will look for certain members on a class.

Since Python is **duck-typed**, there are no interfaces - just implement the members you need.

NOTE: a lot of these methods have top-level functions without underscores you can call to invoke them conveniently
 - eg. instead of m.__hash__(), you can call hash(m)
 - eg. hash(), str(), len(), iter(), etc.

In [1]:
class MyClass:

    def __init__(self):  # Constructor
        pass

    def __str__(self):  # String representation
        return "MyClass"

    def __repr__(self):  # String representation for debugging
        return "MyClass()"

    def __len__(self):  # Length
        return 42

    def __getitem__(self, index):  # Indexing (could be index, key, whatever)
        return index * 2

    def __setitem__(self, index, value):  # Assignment to an index
        pass

    def __delitem__(self, index):  # Deletion of an index
        pass

    def __iter__(self):  # Iteration
        yield 1
        yield 2
        yield 3

    def __contains__(self, item):  # Membership check
        return True

    def __call__(self):  # Callable behavior
        pass

    def __eq__(self, other):  # Equality comparison
        return True

    def __lt__(self, other):  # Less than comparison
        return True

    def __add__(self, other):  # Addition
        return self

    def __sub__(self, other):  # Subtraction
        return self

    def __and__(self, other):  # bitwise &
        return self

    def __bool__(self):  # conversion to boolean
        return True

    def __int__(self):  # conversion to int
        return 0

    def __enter__(self):  # Context manager enter
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):  # Context manager exit
        pass

    def __getattr__(self, name):  # Accessing undefined attribute
        return None

    def __setattr__(self, name, value):  # Setting attribute
        pass

    def __delattr__(self, name):  # Deleting attribute
        pass

    def __getattribute__(self, name):  # Accessing attribute
        return object.__getattribute__(self, name)

    def __hash__(self):  # Hashing
        return hash(self)

    def __enter__(self):  # Context manager enter
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):  # Context manager exit
        pass

    def __del__(self):  # Destructor
        pass
    
    def __new__(cls): # higher-level creation than __init__
        pass

# Context Manager


In [2]:
class MyContext:

    def __init__(self, message):
        self.message = message

    def __enter__(self):
        print('entering')
        return self  # probably the thing that goes to 'as'

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting')
        self.message = None
        if exc_type:
            print(exc_value)
            print(exc_traceback)
            return True  # don't propagate exceptions
        return False  # propagate exceptions


def open_my_context(message):
    return MyContext(message)

In [10]:
# Simple usage of class
with MyContext('hi'):
    pass
print()

# Similar to file I/O API
with open_my_context('hi'):
    pass

entering
exiting

entering
exiting


In [11]:
# Getting a variable for the object
with MyContext('hi') as context:
    print(context.message)
print(context.message)  # variable still exists

entering
hi
exiting
None


In [15]:
# Exception
with MyContext('hi'):
    raise TypeError('uh oh')
    print('unreachable code')

entering
exiting
uh oh


In [4]:
# Multiple contexts without having to nest
with MyContext('c1') as m1, MyContext('c2') as m2:
    print(m1.message)
    print(m2.message)

entering
entering
c1
c2
exiting
exiting


# Iterable


In [5]:
class MyIterator:

    def __init__(self, container):
        self.container = container
        self.index = 0

    def __next__(self):
        if self.index >= len(self.container.data):
            raise StopIteration
        value = self.container.data[self.index]
        self.index += 1
        return value


class MyIterable:

    def __init__(self, data):
        self.data = data

    def __iter__(self):
        return MyIterator(self)


# Usage
my_iterable = MyIterable([1, 2, 3, 4, 5])

for item in my_iterable:
    print(item)
print()

# Lower-level Usage
my_iterable = MyIterable([1, 2, 3, 4, 5])
my_iterator = iter(my_iterable)  # normally the for loop does this
while True:
    try:
        item = next(my_iterator)  # normally the for loop does this
        print(item)
    except StopIteration:  # normally the for loop catches this
        break

1
2
3
4
5

1
2
3
4
5


# Generator

**lazy-loaded**

A generator is a **type of iterable**.


In [6]:
def my_generator():
    print('doing work for 1')
    yield 1  # values provided to next()
    print('doing work for 2')
    yield 2
    print('doing work for 3')
    yield 3
    print('the end')
    return  # ends generation (raises StopIteration)
    yield 4  # never yielded
    # StopIteration would be raised here if no return


generator = my_generator()  # generator function returns iterable
for val in generator:
    print(val)

doing work for 1
1
doing work for 2
2
doing work for 3
3
the end


In [7]:
# Included because a lot of APIs like this pattern
def pass_me_a_generator_function(fn):
    print(list(fn()))


pass_me_a_generator_function(my_generator)

doing work for 1
doing work for 2
doing work for 3
the end
[1, 2, 3]


In [8]:
# Generator comprehension
generator = (i**2 for i in range(3))
for val in generator:
    print(val)

0
1
4


In [9]:
# Convert lazy-loaded generator to in-memory iterable
eager = list(generator)

# Generator from Generator

Iterate over a lazy sequence while **maintaining the laziness**.


In [11]:
def my_generator():
    print('doing work for 1')
    yield 1  # values provided to next()
    print('doing work for 2')
    yield 2
    print('doing work for 3')
    yield 3
    print('the end')
    return  # ends generation (raises StopIteration)
    yield 4  # never yielded


# This would work (and be lazy) even for an eager sequence.
def squares(seq):
    for item in seq:
        yield item**2


gen = my_generator()
sqrs = squares(gen)

print('starting iteration now')
for sqr in sqrs:
    print(sqr)

starting iteration now
doing work for 1
1
doing work for 2
4
doing work for 3
9
the end


# yield statement forms


In [7]:
# We saw this above already.
def simple_generator():
    yield 1
    yield 2
    yield 3


# Included sub-generator items in-place.
def higher_generator():
    yield 0
    yield from simple_generator()
    yield 4


# Yield as part of expression.
def more_stateful_generator():
    x1 = yield 1
    x2 = yield x1**2
    x3 = yield x2**3


print(list(simple_generator()))
print(list(higher_generator()))

gen = more_stateful_generator()
print(next(gen))
print(gen.send(5))  # the x2 assignment gets what you send in


[1, 2, 3]
[0, 1, 2, 3, 4]
1
25


# generator.send()

Send is a way to do two-way communication with a generator. After the first next() to start the iteration (first yield inside the generator), the generator sees the next value sent in. Then it returns some expression based on that, and it goes back and forth like that.

The yield statement sends what it says and returns what it sent back (which may be the same or different).

If you use next() instead of send(), it will send in None.


In [20]:
def two_way_communication_generator():
    x1 = yield 1
    print('x1:', x1)
    x2 = yield x1
    print('x2:', x2)
    x3 = yield x2


try:
    generator = two_way_communication_generator()
    print(next(generator))
    print(generator.send(10))
    print(generator.send(20))

except StopIteration:
    pass

1
x1: 10
10
x2: 20
20


In [22]:
def long_living_generator():
    next_val = 1
    while True:
        next_val = (yield next_val)**2
        if next_val == 0:
            return


try:
    gen = long_living_generator()
    print(next(gen))
    print(gen.send(1))
    print(gen.send(10))
    print(gen.send(2))
    print(gen.send(0))
except StopIteration:
    pass

1
1
100
4


# Generator Comprehension in Function Call

In [24]:
def f(gen):
  for item in gen:
    print(item)
    
f(x**2 for x in range(10))

0
1
4
9
16
25
36
49
64
81


# Properties

The @property decorator makes a method into a property, which means you can access it like an attribute and the getter will be called.

The setter decorator shown below is to implement a setter for a specific property.

Internally, @property taps into the internals of how a class gets and sets attributes.


In [32]:
class Circle:

    def __init__(self, radius):
        self.radius = radius

    @property
    def diameter(self):
        return 2 * self.radius

    @diameter.setter
    def diameter(self, value):
        self.radius = value / 2

    @property
    def area(self):
        return 3.14159 * self.radius**2


circle = Circle(1)
print(circle.diameter)
circle.diameter = 4
print(circle.diameter)
print(circle.area)

2
4.0
12.56636


# Dataclass

This is fairly new (Python 3.7).

It allows you to skip defining a constructor just for the purpose of defining member variables for plain old data types.

Check documentation for some sharp edges around **subclasses** and **custom constructor behavior**.

Note that `__eq__`, `__repr__`, and `__str__` are automatically implemented for data classes as well.

It does not get `__hash__` unless you pass `frozen=True` which makes it __read-only__.

In [10]:
from dataclasses import dataclass


@dataclass
class MyClass:
    x: int  # required constructor arg
    y = 20  # optional constructor arg

    def f(self):
        return self.x + self.y

    # __init__ defined for you
    # has the member fields in order
    # ones with values are optional


m = MyClass(10)
print(m.f())

# Operators
n = MyClass(10)
print(m == n)
print(m)
print(str(m))
# print(hash(m)) # not implemented unless pass frozen=Ture in decorator!

30
True
MyClass(x=10)
MyClass(x=10)


# object

The implicit root of all classes in Python is `object`.  This even applies to __primitives__.

This gives you certain behavior like a hash function that's different for each instance so that you can use your class as dictionary keys right off the bat.

In [5]:
class MyClass:
    pass

print(isinstance(MyClass, object))
print(isinstance(5, object))

m = MyClass()
n = MyClass()

print(m.__hash__())
print(n.__hash__())

o1 = object()
o2 = object()

print(o1.__hash__())
print(o2.__hash__())

True
True
277953588
277953591
276298648
278129751


# Object Methods Used by Collections

The class below traces several operator calls that might happen (not all of them do) in the course of being used as an item in a collection.  It then uses the object in a few collection scenarios.

Summary:
  - `<` operator (__lt__) is used for sorting and searching
    - the insertion point is found when `<` stops being true (might be equal or not)
  - `==` is used for membership tests in non-hashed collections
  - `hash()` and `==` are used for all retrievals in hashed collections
    - only `hash()` is needed for insertion though

In [22]:
from bisect import bisect_left

class MyClass:
    def __init__(self, val):
        self.val = val
    def __lt__(self, other):
        print('< operator on ' + str(self.val))
        return self.val < other.val
    def __gt__(self, other):
        print('> operator on ' + str(self.val))
        return self.val > other.val
    def __eq__(self, other):
        print('== operator on ' + str(self.val))
        return self.val == other.val
    def __hash__(self):
        print('hash on ' + str(self.val))
        return hash(self.val)
    def __str__(self):
        return str(self.val)
    def __repr__(self):
        return repr(self.val)
    
print('original')
l = [MyClass(2), MyClass(1), MyClass(4), MyClass(8), MyClass(2)]
print(l)
print()

print('sorted by key')
s = sorted(l, key=lambda m: m.val) # no operators called
print(s)
print()

print('sorted by <')
s = sorted(l)
print(s)
print()

print('reverse sorted by <')
s = sorted(l, reverse=True)
print(s)
print()

print('membership')
print(MyClass(2) in l)
print()

print('sort again')
s = sorted(l)
print()

print('binary search')
index = bisect_left(s, MyClass(2))
print(s[index])
print()

print('equality')
print(s[index] == MyClass(2))
print(s[index] is MyClass(2))
print()

print('dictionary creation')
d = {MyClass(1): 10, MyClass(2): 20, MyClass(3): 30}
print()

print('dictionary retrieval')
print(d[MyClass(1)])
print()

print('dictionary membership')
print(MyClass(1) in d)
print()

original
[2, 1, 4, 8, 2]

sorted by key
[1, 2, 2, 4, 8]

sorted by <
< operator on 1
< operator on 4
< operator on 4
< operator on 8
< operator on 8
< operator on 2
< operator on 2
[1, 2, 2, 4, 8]

reverse sorted by <
< operator on 8
< operator on 4
< operator on 4
< operator on 4
< operator on 1
< operator on 1
< operator on 2
< operator on 2
[8, 4, 2, 2, 1]

membership
== operator on 2
True

sort again
< operator on 1
< operator on 4
< operator on 4
< operator on 8
< operator on 8
< operator on 2
< operator on 2

binary search
< operator on 2
< operator on 2
< operator on 1
2

equality
== operator on 2
True
False

dictionary creation
hash on 1
hash on 2
hash on 3

dictionary retrieval
hash on 1
== operator on 1
10

dictionary membership
hash on 1
== operator on 1
True



# Extension Methods

Python doesn't directly have extension methods, but you can easily do the same thing by creating a function that takes `self` as the first arg and assigning it as an attribute of the class.

In [4]:
class MyClass:
    def __init__(self, val):
        self.val = val
    def __repr__(self):
        return repr(self.val)
    
def my_extension(self):
    self.val *= 10
    
# adding the method to the class
MyClass.my_extension = my_extension

m = MyClass(1)
print(m)
# call like any other method
m.my_extension()
print(m)

1
10


# Default Constructor

A class can be constructed without an `__init__` method.

But any method called `__init__` will replace any other (due to no overloading in Python).

In [6]:
class MyClass:
    def __repr__(self):
        return ''

class MyOtherClass:
    def __init__(self, x):
        pass
    
class MyOtherOtherClass:
    def __init__(self): # this is uncallable!
        pass
    
    def __init__(self, x):
        pass
    
m = MyClass()
n = MyOtherClass(10)
o = MyOtherOtherClass(10)

# "We Are All Robots"

Private variables work between instances of the same class.

In [7]:
class Robot:
    __x = 5
    
    def printOther(self, other):
        print(other.__x)
        
r1 = Robot()
r2 = Robot()
r1.printOther(r2)

5


# Prototype-Based Inheritance Details

Rather than an instance being a copy of the class data on instantiation, it is __dynamically forwarded to the class__.

Any attributes directly set on an instance after creation will be used directly.  This allows you to add and replace methods and variables.

Others will be forwarded to the class, which will in turn forward to the base class of that, etc. Because it goes backward and left-to-right, that is how diamond inheritance behavior comes about.  It is __dynamic__, rather than being a clone stamp.  If you modify the class's data after instance creation, instances will see updated values.

`super()` is a way to get the superclass of the current class to call a method from the superclass in the derived version.

In [19]:
class BaseClass:
    x = 1
    
    def __init__(self):
        self.y = 1
        self.z = 1
        
    def __str__(self):
        return str(self.x) + ',' + str(self.y) + ',' + str(self.z)
    
    def print_x(self):
        print(self.x)
        
class DerivedClass(BaseClass):
    x = 100
    
    def __init__(self):
        super().__init__()
        self.y = 100

    def print_x(self):
        super().print_x()
        print('sub')
        
m = DerivedClass()
print(str(m))
m.print_x()

n = DerivedClass()
DerivedClass.x = 1000 # affects it
n.print_x()
BaseClass.x = 10000 # doesn't affect it
n.print_x()

m.x = 5
m.print_x()
print(DerivedClass.x)

print(type(m))

100,100,1
100
sub
1000
sub
1000
sub
5
sub
1000
<class '__main__.DerivedClass'>


# Static Methods

In [21]:
class MyClass:
    @staticmethod
    def f():
        print('hi')
        
m = MyClass()
m.f()
MyClass.f()

hi
hi


# Abstract Base Class

In [24]:
from abc import ABC, abstractmethod

class MyInterface(ABC):
    @abstractmethod
    def my_method(self):
        """This is an abstract method that must be implemented in subclasses."""
        pass

class ConcreteClass(MyInterface):
    def my_method(self):
        print("Implementation of my_method")

# Trying to instantiate MyInterface directly will result in an error
# my_interface = MyInterface()  # This would raise TypeError

# Instantiating ConcreteClass which implements the abstract method
concrete_instance = ConcreteClass()
concrete_instance.my_method()  # This will work fine

Implementation of my_method


# Hashing Fields of Objects

Wrap them in a __tuple__ and hash that.

# Uninitialized Class Variables

There is no way to declare a class/instance variable without setting to a value.  You can give a __type hint__ like below, but it's not a real attribute until it's set in the constructor or elsewhere.  It's just a hint that the class will have this member.

Type hints on the class will go into the class's __\_\_annotations\_\___ member, which can be read by decorators like `dataclass` to automatically give behavior to the fields.

In [10]:
class MyClass:
    # x  # ILLEGAL
    x: int # LEGAL, but not real (just type hint)
    
m = MyClass()
# print(m.x)  # fails
# print(MyClass.x) # fails

# Slice Operator

`slice` is actually a built-in top-level data type, automatically insantiated and passed in when you do `a[2:3]` for instance.

So when you overload `__getitem__` and `__setitem__`, you can use `isinstance(key, slice)` to tell if it's a slice vs. an index (`isinstance(key, int)`).

In [6]:
s = slice(2, 3)
print(s)
print(isinstance(s, slice))

slice(2, 3, None)
True


# Multidimensional Indices

When you use commas between slices or indices in an index operator, you are passing a `tuple` to the operator method.

In [2]:
class MyClass:
    def __getitem__(self, index):
        print(index)
        print(type(index))
    
m = MyClass()
m[2:3]
m[2:3,1]

slice(2, 3, None)
<class 'slice'>
(slice(2, 3, None), 1)
<class 'tuple'>


# Right-Side Operators

If your class is on the right side of an operator instead of the left, then the type on the left will be called instead at first.  If that type returns `NotImplemented`, then your class's `__radd__` (for instance, for adding) will be called.